# Assignment 3

In [10]:
import numpy as np
import pandas as pd 
import scipy as sci
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from itertools import chain
from numpy import pi, cos, sin, exp
from scipy.signal import correlate, square, unit_impulse

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

In [4]:
# Import data
D = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/AllStations_temperature_h_2017.dat', 
                 sep='\s+', header=1, usecols=[0,35])

In [5]:
# Convert time in D from MatLab time to Python Time
D['Time'] = D['NaN'].apply(lambda matlab_datenum: 
                             dt.datetime.fromordinal(int(matlab_datenum)) 
                             + dt.timedelta(days=matlab_datenum%1)
                             - dt.timedelta(days = 366)) 

# Rename the columns
D2 = D.rename(index=str, 
                columns={"NaN": "MatLab Time", "48.4623": "Temperature"})

In [6]:
# Reorder columns 
cols = D2.columns.tolist()
cols = cols[-1:] + cols[:-1]
D23 = D2[cols]

# Set time as index column
DH = D23.set_index('Time')

In [7]:
# Select the dates:
# Hour resolution data
DH1 = DH.loc['2015-12-01 00:00':'2016-03-01 23:00']['Temperature']
DH2 = DH.loc['2016-06-01 00:00':'2016-09-01 23:00']['Temperature']

## Question 1: Properties of Fourier transforms 

### b) Fourier transform of a cosine function

$$ F\left\{\cos(\omega_0 a t)\right\} = \frac{1}{2} \left[\delta(af_0 - f) + \delta(af_0 + f)\right]$$

In [9]:
N = 2000
f0 = int(N/4)
alpha1 = 1.5
alpha2 = 0.5
f = np.linspace(-2, 2, N)

f00 = 0.5*(unit_impulse(N, f0) + unit_impulse(N, -f0))
f01 = 0.5*(unit_impulse(N, int(alpha2*f0)) + unit_impulse(N, -int(alpha2*f0)))
f02 = 0.5*(unit_impulse(N, int(alpha1*f0)) + unit_impulse(N, -int(alpha1*f0)))

plt.figure(figsize = (10, 4))
plt.plot(f, f01, label=alpha1)
plt.plot(f, f02, label=alpha2)
plt.plot(f, f00, label=1)

plt.title('Fourier transform of $x(at) = cos(\omega_0 at)$ for $f_0 = 1$ and various values of $a$')
plt.xlabel('$f_0$')

plt.legend()

<Figure size 720x288 with 1 Axes>

### c) One-sided Fourier transform of $e^{-at}$

$$x(t) = 
\begin{cases} 
  0 & t < 0 \\
  e^{-at} & 0 \leq t  
\end{cases}$$ 

$$F\left\{ x(t) \right\} = \left[a^2 + (2\pi f)^2\right]^{-\frac{1}{2}} e^{-i \tan\left(\frac{2\pi f}{a}\right)}$$

In [38]:
def x1c(a, t):
    x1c = exp(-a*t)
    return x1c

def Fx1c(a, t): 
    fx1c = np.fft.fft(x1c(a, t))
    return fx1c


In [156]:
t1c = np.linspace(0, 10, 10000)
fs1c = np.fft.fftfreq(10000, 0.1)
a_Val = 16

fig, (ax1c1, ax1c2, ax1c3, ax1c4, ax1c5) = plt.subplots(5, 1, figsize=(10, 20))

for a in range(0, a_Val, 2):
    ax1c1.plot(t1c, x1c(a/10, t1c), label=a/10)
ax1c1.legend()

ax1c1.set_title('$x(t)=e^{-at}$ for various values of $a$')
ax1c1.set_xlabel('Time (s)')
ax1c1.set_ylabel('$x(t)$')

for a in range(0, a_Val, 2):
    ax1c2.semilogy(fs1c, Fx1c(a, t1c).real, label=a/10)
    ax1c3.plot(fs1c, Fx1c(a, t1c).imag, label=a/10)
    ax1c4.semilogy(fs1c, np.abs(Fx1c(a, t1c)), label=a/10)
    ax1c5.plot(fs1c, np.angle(Fx1c(a, t1c)), label=a/10)

ax1c2.set_ylim(0.1, 0.2*10**5)
ax1c2.set_xlim(-2, 2)
ax1c2.legend()

ax1c3.set_xlim(-0.2, 0.2)
ax1c3.legend(loc=1)

ax1c4.set_ylim(0.1, 0.2*10**5)
ax1c4.legend()

ax1c5.set_ylim(-1.75, 1.75)
ax1c5.legend(loc=1)

ax1c2.set_title('Real part of the Fourier transform of $x(t)=e^{-at}$ for various values of $a$')
ax1c2.set_xlabel('Frequency ($Hz$)')
ax1c2.set_ylabel('$Re[X(f)]$')

ax1c3.set_title('Imaginary part of the Fourier transform of $x(t)=e^{-at}$ for various values of $a$')
ax1c3.set_xlabel('Frequency ($Hz$)')
ax1c3.set_ylabel('$Im[X(f)]$')

ax1c4.set_title('Magnitude of the Fourier transform of $x(t)=e^{-at}$ for various values of $a$')
ax1c4.set_xlabel('Frequency ($Hz$)')
ax1c4.set_ylabel('$| X(f) |$')

ax1c5.set_title('Phase of the Fourier transform of $x(t)=e^{-at}$ for various values of $a$')
ax1c5.set_xlabel('Frequency ($Hz$)')
ax1c5.set_ylabel('$\Theta (f)$')

fig.tight_layout()

<Figure size 720x1440 with 5 Axes>

As expected, the phase $\theta(f) = -\tan\left(\frac{2\pi f}{a}\right)$ and the magnitude $\left| X(f) \right| = \left[a^2 + (2\pi f)^2\right]^{-\frac{1}{2}}$ decreased with the increase in $a$.

## Question 2: Quality of dummy weather forcast

### 1) Winter data set:

In [151]:
plt.figure(figsize = (15, 6))
ax1 = DH1.plot(label = 'Temperature')
plt.title('Temperature data from UVic Sci from 1 Dec 2015 to 1 Mar 2016', fontsize=16)
plt.xlabel('Time (hr)', fontsize=14)
plt.ylabel('Temperature ($^\circ C$)', fontsize=14)
ax1.xaxis.set_major_locator(mp.dates.DayLocator(bymonthday = (1, 8, 15, 22)))
plt.show()

<Figure size 1080x432 with 1 Axes>

#### Correlation

In [142]:
correlate(DH1, DH1)

array([ 42.9226,  82.1312, 122.9707, ..., 122.9707,  82.1312,  42.9226])

### 2) Summer data set:

In [150]:
plt.figure(figsize = (15, 6))
ax2 = DH2.plot(label = 'Temperature')
plt.title('Temperature data from UVic Sci from 1 Jun 2016 to 1 Sep 2016',  fontsize=16)
plt.xlabel('Time (hr)', fontsize=14)
plt.ylabel('Temperature ($^\circ C$)', fontsize=14)
ax2.xaxis.set_major_locator(mp.dates.DayLocator(bymonthday = (1, 8, 15, 22)))
plt.show()

<Figure size 1080x432 with 1 Axes>

# Question 3: Fourier series of a square wave

$$x(t) = 
\begin{cases} 
  -1 & -\frac{T}{2} < t < 0 \\
  1 & 0 \leq t < \frac{T}{2} 
\end{cases} 
\ \ \ \ \ \ \ \ \
x(t+nT) = x(t)$$

We found that:

$$x(t) \sim \frac{1}{2} \sum_{m=-\infty}^{\infty} S_m e^{i\left(\frac{m\pi}{T} \right) t}$$

$$S_m = \frac{i}{m\pi} \left[ 1 - \cos\left(m\pi\right)\right] =
\begin{cases} 
  \frac{2i}{m\pi} & \text{for } m \text{ odd} \\
  0 & \text{for } m \text{ even}
\end{cases}$$

We can reduce this into a Fourier sine series, which makes calulations easier:

$$x(t) \sim 2\sum_{n=1}^{\infty} b_m \sin\left(\frac{m\pi}{T} t \right)$$

$$b_m = \frac{1}{m\pi} \left[ 1 - \cos\left(m\pi\right)\right] =\frac{1}{m\pi} \left[ 1 - (-1)^m\right]
\begin{cases} 
  \frac{2}{m\pi} & \text{for } m \text{ odd} \\
  0 & \text{for } m \text{ even}
\end{cases}$$

Let's plot the graph as a function of T

100